In [ ]:
import cv2
import numpy as np
import json
import os
import random
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Reshape, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
#from keras.applications.resnet50 import ResNet50
from keras.applications.resnet import ResNet50
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import pandas as pd
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import seaborn as sns
import shutil
from PIL import Image
from matplotlib import image
from sklearn.cluster import AgglomerativeClustering
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, Model, Input

In [ ]:
!pip install tensorflow-addons
import tensorflow_addons as tfa

     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [ ]:
! pip install imagehash

In [ ]:
# Load data

X_train = np.load('/content/drive/MyDrive/CV_Images_npy_11/X_train.npy')
y_train = np.load('/content/drive/MyDrive/CV_Images_npy_11/y_train.npy')
X_test_cr = np.load('/content/drive/MyDrive/CV_Images_npy_11/X_test_cr.npy')
y_test_cr = np.load('/content/drive/MyDrive/CV_Images_npy_11/y_test_cr.npy')
X_test_uncr = np.load('/content/drive/MyDrive/CV_Images_npy_11/X_test_uncr.npy')
y_test_uncr = np.load('/content/drive/MyDrive/CV_Images_npy_11/y_test_uncr.npy')

In [ ]:
He_initializer = tf.keras.initializers.HeNormal()

width_shape = 224
height_shape = 224
image_input = Input(shape=(width_shape, height_shape, 3))

def getEfficientNetModel(lastFourTrainable=False):
  
  eff_model = EfficientNetB0(include_top=False, input_tensor=image_input, weights="imagenet")

  # Make all layers untrainable
  for layer in eff_model.layers[:]:
      layer.trainable = False

  # Add fully connected layer which have 1024 neuron to VGG-16 model
  output = eff_model.get_layer('top_activation').output
  output = Flatten(name='new_flatten')(output)
  output = Dense(units=1024, activation='relu', name='new_fc',kernel_initializer= He_initializer)(output)
  output = Dense(units=11, activation='softmax',kernel_initializer= He_initializer)(output)
  eff_model = Model(eff_model.input, output)

  #Make last 4 layers trainable if lastFourTrainable == True
  if lastFourTrainable == True:
    eff_model.get_layer('top_conv').trainable = True
    #eff_model.get_layer('block7a_project_conv').trainable = True
    
    #vgg_model.get_layer('fc1').trainable = True
    #vgg_model.get_layer('fc2').trainable = True
    eff_model.get_layer('new_fc').trainable = True
    
  # to run discriminative learning uncomment below
  # optimizers = [
  #    tf.keras.optimizers.Adam(learning_rate=1e-6),
  #    tf.keras.optimizers.Adam(learning_rate=1e-3)]

  # optimizers_and_layers = [(optimizers[1], eff_model.layers[-1]),(optimizers[1], eff_model.layers[-2]),
  #                           (optimizers[0], eff_model.layers[-6])]#, (optimizers[0], eff_model.layers[-8])]

  # optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

  # eff_model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

  eff_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  # eff_model.summary()
  return eff_model

# Without Augmentation

In [ ]:
eff_model =  getEfficientNetModel()

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True) 
eff_model.fit(X_train, y_train, epochs=25, validation_data=(X_test_uncr, y_test_uncr), batch_size=50, callbacks=[es])

Epoch 1/25
18/18 [==============================] - 26s 528ms/step - loss: 6.8585 - accuracy: 0.6883 - val_loss: 20.1330 - val_accuracy: 0.1933
Epoch 2/25
18/18 [==============================] - 5s 297ms/step - loss: 0.3339 - accuracy: 0.9681 - val_loss: 9.3863 - val_accuracy: 0.5951
Epoch 3/25
18/18 [==============================] - 5s 302ms/step - loss: 0.0445 - accuracy: 0.9954 - val_loss: 6.5059 - val_accuracy: 0.6074
Epoch 4/25
18/18 [==============================] - 5s 286ms/step - loss: 2.2565e-04 - accuracy: 1.0000 - val_loss: 6.3614 - val_accuracy: 0.5920
Epoch 5/25
18/18 [==============================] - 5s 284ms/step - loss: 0.0100 - accuracy: 0.9977 - val_loss: 6.2468 - val_accuracy: 0.5429
Epoch 6/25
18/18 [==============================] - 5s 284ms/step - loss: 5.1804e-04 - accuracy: 1.0000 - val_loss: 6.7072 - val_accuracy: 0.5153
Epoch 7/25
18/18 [==============================] - 5s 285ms/step - loss: 0.0079 - accuracy: 0.9977 - val_loss: 6.4771 - val_accuracy: 0.5

In [ ]:
y_pred_cr = eff_model.predict(X_test_cr)
y_pred_uncr = eff_model.predict(X_test_uncr)

_, acc = eff_model.evaluate(X_test_cr, y_test_cr, verbose=0)
print('> %.3f' % (acc * 100.0))

_, acc = eff_model.evaluate(X_test_uncr, y_test_uncr, verbose=0)
print('> %.3f' % (acc * 100.0))

> 69.325
> 60.736


# Augmentation 1 (Default)

In [ ]:
X_train_dg = ImageDataGenerator()
X_train_dg.fit(X_train)

In [ ]:
eff_model =  getEfficientNetModel()

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True) 
eff_model.fit(X_train_dg.flow(X_train, y_train), epochs=25, validation_data=(X_test_uncr, y_test_uncr), batch_size=50, callbacks=[es])

Epoch 1/25
28/28 [==============================] - 16s 274ms/step - loss: 9.8205 - accuracy: 0.7088 - val_loss: 9.7318 - val_accuracy: 0.5000
Epoch 2/25
28/28 [==============================] - 5s 194ms/step - loss: 0.1732 - accuracy: 0.9784 - val_loss: 7.6311 - val_accuracy: 0.4540
Epoch 3/25
28/28 [==============================] - 5s 194ms/step - loss: 0.0094 - accuracy: 0.9966 - val_loss: 5.9785 - val_accuracy: 0.5000
Epoch 4/25
28/28 [==============================] - 5s 196ms/step - loss: 0.0055 - accuracy: 0.9966 - val_loss: 7.0701 - val_accuracy: 0.4448
Epoch 5/25
28/28 [==============================] - 6s 204ms/step - loss: 2.1594e-06 - accuracy: 1.0000 - val_loss: 5.6752 - val_accuracy: 0.5123
Epoch 6/25
28/28 [==============================] - 6s 203ms/step - loss: 7.4814e-06 - accuracy: 1.0000 - val_loss: 5.6283 - val_accuracy: 0.5245
Epoch 7/25
28/28 [==============================] - 6s 198ms/step - loss: 2.2660e-05 - accuracy: 1.0000 - val_loss: 5.6179 - val_accuracy: 

In [ ]:
y_pred_cr = eff_model.predict(X_test_cr)
y_pred_uncr = eff_model.predict(X_test_uncr)

_, acc = eff_model.evaluate(X_test_cr, y_test_cr, verbose=0)
print('> %.3f' % (acc * 100.0))

_, acc = eff_model.evaluate(X_test_uncr, y_test_uncr, verbose=0)
print('> %.3f' % (acc * 100.0))

> 66.564
> 52.761


# Augmentation 2

In [ ]:
X_train_dg = ImageDataGenerator(horizontal_flip=True)
X_train_dg.fit(X_train)

In [ ]:
eff_model =  getEfficientNetModel()

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True) 
eff_model.fit(X_train_dg.flow(X_train, y_train), epochs=25, validation_data=(X_test_uncr, y_test_uncr), batch_size=50, callbacks=[es])

Epoch 1/25
28/28 [==============================] - 16s 283ms/step - loss: 7.6913 - accuracy: 0.6871 - val_loss: 4.8471 - val_accuracy: 0.5276
Epoch 2/25
28/28 [==============================] - 6s 206ms/step - loss: 0.2512 - accuracy: 0.9693 - val_loss: 2.8861 - val_accuracy: 0.6871
Epoch 3/25
28/28 [==============================] - 5s 197ms/step - loss: 0.1055 - accuracy: 0.9841 - val_loss: 4.1271 - val_accuracy: 0.6472
Epoch 4/25
28/28 [==============================] - 5s 196ms/step - loss: 0.0440 - accuracy: 0.9863 - val_loss: 4.4000 - val_accuracy: 0.6380
Epoch 5/25
28/28 [==============================] - 6s 200ms/step - loss: 0.0431 - accuracy: 0.9909 - val_loss: 4.3808 - val_accuracy: 0.6656
Epoch 6/25
28/28 [==============================] - 6s 203ms/step - loss: 0.0613 - accuracy: 0.9875 - val_loss: 4.7222 - val_accuracy: 0.6350
Epoch 7/25
28/28 [==============================] - 6s 207ms/step - loss: 0.0237 - accuracy: 0.9943 - val_loss: 5.4619 - val_accuracy: 0.6534


In [ ]:
y_pred_cr = eff_model.predict(X_test_cr)
y_pred_uncr = eff_model.predict(X_test_uncr)

_, acc = eff_model.evaluate(X_test_cr, y_test_cr, verbose=0)
print('> %.3f' % (acc * 100.0))

_, acc = eff_model.evaluate(X_test_uncr, y_test_uncr, verbose=0)
print('> %.3f' % (acc * 100.0))

> 73.313
> 68.712


# Augmentation 3

In [ ]:
X_train_dg = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
X_train_dg.fit(X_train)

In [ ]:
eff_model =  getEfficientNetModel()

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True) 
eff_model.fit(X_train_dg.flow(X_train, y_train), epochs=25, validation_data=(X_test_uncr, y_test_uncr), batch_size=50, callbacks=[es])

Epoch 1/25
28/28 [==============================] - 24s 605ms/step - loss: 26.8540 - accuracy: 0.1229 - val_loss: 11.6756 - val_accuracy: 0.0859
Epoch 2/25
28/28 [==============================] - 13s 459ms/step - loss: 4.1450 - accuracy: 0.1206 - val_loss: 11.3341 - val_accuracy: 0.0859
Epoch 3/25
28/28 [==============================] - 13s 459ms/step - loss: 2.6434 - accuracy: 0.1570 - val_loss: 11.2887 - val_accuracy: 0.0859
Epoch 4/25
28/28 [==============================] - 14s 493ms/step - loss: 2.5301 - accuracy: 0.1445 - val_loss: 11.3508 - val_accuracy: 0.0828
Epoch 5/25
28/28 [==============================] - 13s 463ms/step - loss: 2.4480 - accuracy: 0.1752 - val_loss: 11.7726 - val_accuracy: 0.0859
Epoch 6/25
28/28 [==============================] - 12s 441ms/step - loss: 2.7153 - accuracy: 0.1377 - val_loss: 12.0902 - val_accuracy: 0.0859


In [ ]:
y_pred_cr = eff_model.predict(X_test_cr)
y_pred_uncr = eff_model.predict(X_test_uncr)

_, acc = eff_model.evaluate(X_test_cr, y_test_cr, verbose=0)
print('> %.3f' % (acc * 100.0))

_, acc = eff_model.evaluate(X_test_uncr, y_test_uncr, verbose=0)
print('> %.3f' % (acc * 100.0))

> 6.748
> 8.589


# Augmentation 4

In [ ]:
X_train_dg = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
X_train_dg.fit(X_train)

In [ ]:
eff_model =  getEfficientNetModel()

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True) 
eff_model.fit(X_train_dg.flow(X_train, y_train), epochs=25, validation_data=(X_test_uncr, y_test_uncr), batch_size=50, callbacks=[es])

Epoch 1/25
28/28 [==============================] - 21s 496ms/step - loss: 10.1789 - accuracy: 0.6052 - val_loss: 5.6265 - val_accuracy: 0.4908
Epoch 2/25
28/28 [==============================] - 13s 458ms/step - loss: 0.3292 - accuracy: 0.9431 - val_loss: 1.9508 - val_accuracy: 0.7485
Epoch 3/25
28/28 [==============================] - 12s 424ms/step - loss: 0.0863 - accuracy: 0.9852 - val_loss: 1.8650 - val_accuracy: 0.7699
Epoch 4/25
28/28 [==============================] - 12s 418ms/step - loss: 0.1887 - accuracy: 0.9704 - val_loss: 3.6522 - val_accuracy: 0.6871
Epoch 5/25
28/28 [==============================] - 12s 418ms/step - loss: 0.1739 - accuracy: 0.9738 - val_loss: 2.4965 - val_accuracy: 0.7209
Epoch 6/25
28/28 [==============================] - 12s 420ms/step - loss: 0.0432 - accuracy: 0.9909 - val_loss: 2.7531 - val_accuracy: 0.7055
Epoch 7/25
28/28 [==============================] - 12s 430ms/step - loss: 0.0506 - accuracy: 0.9863 - val_loss: 2.1862 - val_accuracy: 0.785

In [ ]:
y_pred_cr = eff_model.predict(X_test_cr)
y_pred_uncr = eff_model.predict(X_test_uncr)

_, acc = eff_model.evaluate(X_test_cr, y_test_cr, verbose=0)
print('> %.3f' % (acc * 100.0))

_, acc = eff_model.evaluate(X_test_uncr, y_test_uncr, verbose=0)
print('> %.3f' % (acc * 100.0))

> 96.319
> 78.528


# Augmentation 4 with Discrim Learning

In [ ]:
def getEfficientNetModel(lastFourTrainable=False):
  
  eff_model = EfficientNetB0(include_top=False, input_tensor=image_input, weights="imagenet")

  # Make all layers untrainable
  for layer in eff_model.layers[:]:
      layer.trainable = False

  # Add fully connected layer which have 1024 neuron to VGG-16 model
  output = eff_model.get_layer('top_activation').output
  output = Flatten(name='new_flatten')(output)
  output = Dense(units=1024, activation='relu', name='new_fc',kernel_initializer= He_initializer)(output)
  output = Dense(units=11, activation='softmax',kernel_initializer= He_initializer)(output)
  eff_model = Model(eff_model.input, output)

  #Make last 4 layers trainable if lastFourTrainable == True
  if lastFourTrainable == True:
    # eff_model.get_layer('top_conv').trainable = True
    eff_model.get_layer('block7a_project_conv').trainable = True
    
    #vgg_model.get_layer('fc1').trainable = True
    #vgg_model.get_layer('fc2').trainable = True
    eff_model.get_layer('new_fc').trainable = True
    
  # to run discriminative learning uncomment below
  optimizers = [
     tf.keras.optimizers.Adam(learning_rate=1e-6),
     tf.keras.optimizers.Adam(learning_rate=1e-3)]

  optimizers_and_layers = [(optimizers[1], eff_model.layers[-1]),(optimizers[1], eff_model.layers[-2]),
                            (optimizers[0], eff_model.layers[-6])]#, (optimizers[0], eff_model.layers[-8])]

  optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

  eff_model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

  # eff_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  # eff_model.summary()
  return eff_model

In [ ]:
X_train_dg = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
X_train_dg.fit(X_train)

In [ ]:
eff_model =  getEfficientNetModel(lastFourTrainable=True)

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True) 
eff_model.fit(X_train_dg.flow(X_train, y_train), epochs=25, validation_data=(X_test_uncr, y_test_uncr), batch_size=50, callbacks=[es])

Epoch 1/25
28/28 [==============================] - 21s 503ms/step - loss: 8.9391 - accuracy: 0.5745 - val_loss: 3.7567 - val_accuracy: 0.5675
Epoch 2/25
28/28 [==============================] - 12s 425ms/step - loss: 0.2557 - accuracy: 0.9556 - val_loss: 3.0360 - val_accuracy: 0.6810
Epoch 3/25
28/28 [==============================] - 12s 428ms/step - loss: 0.1295 - accuracy: 0.9670 - val_loss: 2.5463 - val_accuracy: 0.7055
Epoch 4/25
28/28 [==============================] - 12s 419ms/step - loss: 0.1156 - accuracy: 0.9784 - val_loss: 3.4190 - val_accuracy: 0.6840
Epoch 5/25
28/28 [==============================] - 12s 422ms/step - loss: 0.1101 - accuracy: 0.9795 - val_loss: 3.8580 - val_accuracy: 0.7423
Epoch 6/25
28/28 [==============================] - 12s 417ms/step - loss: 0.1539 - accuracy: 0.9807 - val_loss: 3.7730 - val_accuracy: 0.5767
Epoch 7/25
28/28 [==============================] - 12s 416ms/step - loss: 0.0951 - accuracy: 0.9829 - val_loss: 2.5955 - val_accuracy: 0.7055

In [ ]:
y_pred_cr = eff_model.predict(X_test_cr)
y_pred_uncr = eff_model.predict(X_test_uncr)

_, acc = eff_model.evaluate(X_test_cr, y_test_cr, verbose=0)
print('> %.3f' % (acc * 100.0))

_, acc = eff_model.evaluate(X_test_uncr, y_test_uncr, verbose=0)
print('> %.3f' % (acc * 100.0))

> 85.890
> 74.233
